In [ ]:
import pandas as pd

from pathlib import Path
from cocoviz import ProblemDescription, Result, ResultSet, ertplot, ertvalues

DATA_DIR = Path("data/")

# Read demo results

In [ ]:
results = ResultSet()
for result_file in DATA_DIR.glob("*.csv"):
    # Read CSV file
    data = pd.read_csv(result_file)
    
    # Decode algorithm name and problem description from file name
    algorithm, problem_class, n_dim, run = result_file.stem.split("_")
    n_dim = int(n_dim.split("=")[1])
    run = int(run.split("=")[1])    
    problem = ProblemDescription(problem_class, 1, n_dim, 2)

    # Store results in result set
    results.append(Result(algorithm, problem, data, "Evaluations"))

# Create an ERT plot each problem in the result set

In [ ]:
number_of_targets = 101
indicator = "Hypervolume"

for problem, result_subset in results.by_problem():    
    ax = ertplot(result_subset, "Hypervolume", number_of_targets=number_of_targets)
    ax.set_title(problem)

# Compute ERT values

In [ ]:
evs = {}
for problem, ss in results.by_problem():
    evs[problem] = ertvalues(ss, "Hypervolume")    